# モデルの精度を確認するためのインターフェース

ver.2023/01/10

## 1 概要
「Train_MMVC.ipynb」で学習したモデルでTTSと非リアルタイムのVCを行い、モデルの精度を検証します。

## 2 Google Drive をマウント
**Google Drive にアップロードした MMVC_Trainer を参照できるように、設定します。**

「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」

といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

成功すれば、下記メッセージが出ます。
```
Mounted at /content/drive/
```


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

cdコマンドを実行して、マウントしたGoogle Drive のMMVC_Trainerディレクトリに移動します。

%cd 「MMVC_Trainerをgoogle driveにパップロードしたパス」

としてください。

正しいパスが指定されていれば

-rw------- 1 root root 11780 Mar  4 16:53 attentions.py

-rw------- 1 root root  4778 Mar  4 16:53 commons.py

drwx------ 2 root root  4096 Mar  5 15:20 configs

...といった感じに表示されるはずです。

In [ ]:
%cd /content/drive/MyDrive/MMVC_val/MMVC_Trainer-1.5.0.0_SiFiGAN
!ls -la

## 3 必要なライブラリのインストール

何も考えず実行してください。

In [ ]:
!apt-get install espeak
!pip install -r requirements.txt

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from scipy.io.wavfile import write

## 4 学習したモデルを読み込む

CONFIG_PATH に学習に利用したjsonファイルを`「./configs/****.json」`のように指定し、  
NET_PATHに学習したモデルを`「./configs/xxxx/G_*****.pth」`のように指定してください。


CONFIG_PATH = "./configs/train_config_zundamon.json"  
CONFIG_PATH = "./configs/train_config.json"



特に設定をいじっていない場合、CONFIG_PATHはどちらかになると思います。

In [ ]:
CONFIG_PATH = "./configs/train_config.json"
NET_PATH = "./logs/test1/G_best.pth"

指定したファイルをもとにモデルの読み込みを行います。

In [ ]:
hps = utils.get_hparams_from_file(CONFIG_PATH)

In [ ]:
net_g = SynthesizerTrn(
      spec_channels = hps.data.filter_length // 2 + 1,
      segment_size = hps.train.segment_size // hps.data.hop_length,
      inter_channels = hps.model.inter_channels,
      hidden_channels = hps.model.hidden_channels,
      upsample_rates = hps.model.upsample_rates,
      upsample_initial_channel = hps.model.upsample_initial_channel,
      upsample_kernel_sizes = hps.model.upsample_kernel_sizes,
      n_flow = hps.model.n_flow,
      dec_out_channels=1,
      dec_kernel_size=7,
      n_speakers = hps.data.n_speakers,
      gin_channels = hps.model.gin_channels,
      requires_grad_pe = hps.requires_grad.pe,
      requires_grad_flow = hps.requires_grad.flow,
      requires_grad_text_enc = hps.requires_grad.text_enc,
      requires_grad_dec = hps.requires_grad.dec
      )
_ = net_g.eval()

_ = utils.load_checkpoint(NET_PATH, net_g, True, None)

## 5 学習したモデルで非リアルタイムVCを行う

非リアルタイムのVCを行います。

ソース話者のIDとその話者の音声ファイルのパス、変換ターゲットの話者のIDを指定してください。

In [ ]:
SOURCE_WAVFILE = "/content/drive/MyDrive/MMVC_val/MMVC_Trainer-1.5.0.0_SiFiGAN/dataset/001_jvs001/wav/VOICEACTRESS100_001.wav"
SOURCE_SPEAKER_ID = 1
TARGET_ID = 104
DUMMY = "/content/drive/MyDrive/MMVC_val/MMVC_Trainer-1.5.0.0_SiFiGAN/units/001_jvs001/VOICEACTRESS100_001.npy"
F0 = "/content/drive/MyDrive/MMVC_val/MMVC_Trainer-1.5.0.0_SiFiGAN/F0/001_jvs001/VOICEACTRESS100_001.npy"
CF0 = "/content/drive/MyDrive/MMVC_val/MMVC_Trainer-1.5.0.0_SiFiGAN/cF0/001_jvs001/VOICEACTRESS100_001.npy"
#ここで変換後の音声のF0(ピッチ)を調整
f0_scale = 2.0

実際にVCを行います。

ここでの性能が悪い場合、学習不足か他に問題があります。

In [ ]:
with torch.no_grad():
    dataset = TextAudioSpeakerLoader(hps.data.training_files_notext, hps.data)
    data = dataset.get_audio_text_speaker_pair([SOURCE_WAVFILE, SOURCE_SPEAKER_ID, DUMMY, F0, CF0])
    data = TextAudioSpeakerCollate(
      sample_rate = hps.data.sampling_rate,
      segment_size = hps.train.segment_size,
      hop_size = hps.data.hop_length,
      df_f0_type = hps.data.df_f0_type,
      dense_factors = hps.data.dense_factors,
      upsample_scales = hps.model.upsample_rates,
      sine_amp = hps.data.sine_amp,
      noise_amp = hps.data.noise_amp,
      sine_f0_type = hps.data.sine_f0_type,
      signal_types = hps.data.signal_types,
      train = False,
      f0_factor = f0_scale,
  )([data])
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src, f0, f0_lengths, sin, d, slice_id = [x for x in data]
    sid_tgt1 = torch.LongTensor([TARGET_ID])
    audio1 = net_g.voice_conversion(spec, spec_lengths, sin, d, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate))